# RAGAS 테스트 04 - 성능 비교 분석

이 노트북은 기존 RAG 시스템과 LangGraph RAG 시스템의 성능을 비교 분석합니다.


In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
from datasets import Dataset

print("RAGAS 테스트 04 - 성능 비교 분석 시작")


In [ ]:
# 두 시스템의 API 테스트 함수
def test_original_rag_api(question):
    """기존 RAG API 테스트"""
    url = "http://localhost:8001/api/v1/rag"
    payload = {"prompt": question}
    
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            data = response.json()
            return {
                "answer": data.get("answer", ""),
                "sources": data.get("sources", [])
            }
        else:
            return {"error": f"API 호출 실패: {response.status_code}"}
    except Exception as e:
        return {"error": f"API 호출 중 오류: {str(e)}"}

def test_langgraph_rag_api(question):
    """LangGraph RAG API 테스트"""
    url = "http://localhost:8001/api/v1/langgraph/multiturn"
    payload = {"prompt": question}
    
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            data = response.json()
            return {
                "answer": data.get("response", ""),
                "sources": data.get("sources", [])
            }
        else:
            return {"error": f"API 호출 실패: {response.status_code}"}
    except Exception as e:
        return {"error": f"API 호출 중 오류: {str(e)}"}

# 비교 테스트 질문들
comparison_questions = [
    "KB 동반성장협약 상생대출이 무엇인가요?",
    "KB 스마트론 대출 조건은 무엇인가요?",
    "KB 닥터론 신청 방법을 알려주세요",
    "KB 햇살론은 어떤 상품인가요?",
    "KB 전세대출 금리는 어떻게 되나요?"
]

print(f"비교 테스트 질문 {len(comparison_questions)}개 준비 완료")


In [ ]:
# 두 시스템 테스트 실행
original_results = []
langgraph_results = []

print("기존 RAG 시스템 테스트 중...")
for question in comparison_questions:
    result = test_original_rag_api(question)
    original_results.append({
        "question": question,
        "answer": result.get("answer", ""),
        "sources": result.get("sources", [])
    })

print("LangGraph RAG 시스템 테스트 중...")
for question in comparison_questions:
    result = test_langgraph_rag_api(question)
    langgraph_results.append({
        "question": question,
        "answer": result.get("answer", ""),
        "sources": result.get("sources", [])
    })

print("두 시스템 테스트 완료")


In [ ]:
# RAGAS 평가 실행
def evaluate_system(results, system_name):
    """시스템별 RAGAS 평가"""
    ragas_data = {
        "question": [r["question"] for r in results],
        "answer": [r["answer"] for r in results],
        "contexts": [[source.get("text", "") for source in r["sources"]] for r in results],
        "ground_truths": [
            ["KB 동반성장협약 상생대출은 중소기업 지원 대출"],
            ["KB 스마트론은 온라인 간편 대출"],
            ["KB 닥터론은 의료진 전용 대출"],
            ["KB 햇살론은 희망복지 대출"],
            ["KB 전세대출은 전세보증금 대출"]
        ]
    }
    
    dataset = Dataset.from_dict(ragas_data)
    result = evaluate(
        dataset,
        metrics=[
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall
        ]
    )
    
    return result

print("기존 RAG 시스템 RAGAS 평가 중...")
original_eval = evaluate_system(original_results, "기존 RAG")

print("LangGraph RAG 시스템 RAGAS 평가 중...")
langgraph_eval = evaluate_system(langgraph_results, "LangGraph RAG")

print("두 시스템 평가 완료")


In [ ]:
# 성능 비교 분석
print("\\n=== 성능 비교 분석 결과 ===")

# 결과 비교
comparison_data = {
    "Metric": ["Faithfulness", "Answer Relevancy", "Context Precision", "Context Recall"],
    "기존 RAG": [
        original_eval["faithfulness"],
        original_eval["answer_relevancy"],
        original_eval["context_precision"],
        original_eval["context_recall"]
    ],
    "LangGraph RAG": [
        langgraph_eval["faithfulness"],
        langgraph_eval["answer_relevancy"],
        langgraph_eval["context_precision"],
        langgraph_eval["context_recall"]
    ]
}

df_comparison = pd.DataFrame(comparison_data)
print(df_comparison)

# 시각화
plt.figure(figsize=(12, 8))
df_melted = df_comparison.melt(id_vars=['Metric'], var_name='System', value_name='Score')
sns.barplot(data=df_melted, x='Metric', y='Score', hue='System')
plt.title('RAG 시스템 성능 비교')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
